In [ ]:
%load_ext autoreload
%autoreload 2
from common import *

In [ ]:
RESULT_JSON = "/Users/law/repos/viper/results/kv_size/all_kv_size_revision.json"

In [ ]:
FIXTURES = [VIPER, DASH, HYBRID_FASTER, PMEMKV, CRL_STORE]
BM_TYPES = ['insert', 'get']

BMS = get_all_runs(RESULT_JSON)
# pprint(BMS) 

KV_SIZE_RE = re.compile(r".*KeyType(\d+),ValueType(\d+).*")

def get_kv_size(bm_name):
    match = KV_SIZE_RE.match(bm_name)
    if match is None:
        raise f"unknown kv size: {bm_name}"
    return (int(match.group(1)), int(match.group(2)))

RUNS = defaultdict(list)

for bm in BMS:
    for f, _ in FIXTURES:
        if f in bm['name']:            
            bm_type = get_bm_type(bm['name'])
            kv_size = get_kv_size(bm['name'])
            if kv_size == (16, 100): continue
            RUNS[(f, bm_type)].append((kv_size, bm))
        
    for _, run in RUNS.items():
        run.sort(key=lambda x: x[0]) 
        
# pprint(RUNS)

In [ ]:
fig, (insert_ax, get_ax) = plt.subplots(1, 2, figsize=(10, 3))


def get_bm_ax(bm_type):
    if bm_type == 'insert': return insert_ax
    if bm_type == 'get': return get_ax
    raise RuntimeError(f"Unknown bm_type {bm_type}")

sizes = [16, 116, 216, 1000]
sizes = [16, 216, 532, 1000]
# sizes = [1, 2, 3, 4]

for bm_type, letter in zip(BM_TYPES, ['a', 'b']):
    ax = get_bm_ax(bm_type)
    ax.set_title(f"({letter}) {bm_type.upper()}", fontsize=20)
    for (fixture, _) in FIXTURES:
        r = [run[1]['items_per_second'] / MILLION for run in RUNS[(fixture, bm_type)]]
        style = STYLES[fixture]
        x_vals = sizes[1:] if fixture == PMEMKV[0] or fixture == CRL_STORE[0] else sizes
        ax.plot(x_vals, r, marker=style.marker, ms=8, #ms=style.marker_size, 
                color=style.color, markeredgewidth=0.5, lw=2)
    ax.set_xticks(sizes)
    ax.set_xticklabels([str(x) for x in sizes])
    ax.set_xticklabels(['8/8', '16/200', '32/500', '100/900'])
    ax.grid(axis='y', which='major')
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(19)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(19)


insert_ax.set_ylabel("Throughput (Mops/s)", fontsize=18)
insert_ax.set_title(f"(a) PUT", fontsize=20)
insert_ax.set_ylim(0, 22)
get_ax.set_ylim(0, 45)

fig.text(0.542, -0.05, "Key/Value Record Size in Byte", ha='center', fontsize=20)

fig.legend(loc='upper center', labels=[f[1] for f in FIXTURES], 
           bbox_to_anchor=(0.5, 1.13), ncol=6, frameon=False, fontsize=18,
           columnspacing=1, handletextpad=0.3)

plt.tight_layout()
for ax in (insert_ax, get_ax):
    hide_border(ax, True)
    
fig.savefig('charts/kv_size.pdf', bbox_inches='tight')
fig.savefig('charts/kv_size.svg', bbox_inches='tight')